In [1]:
import requests
import json
import pandas as pd
import csv
from pandas import json_normalize
from datetime import datetime, date, timedelta
from pandas.tseries.offsets import DateOffset

In [ ]:
import requests
import json
import pandas as pd
import csv
from pandas import json_normalize
from datetime import datetime, date, timedelta
from pandas.tseries.offsets import DateOffset

#Query Details that is passed in the URL
limit = '50'
offset = '50'
total_count = 'true'

#Create an empty list
all_items = []


def get():



    url_address="https://onesignal.com/api/v1/notifications?app_id={enter-app-id-here}&kind=0&limit=0&offset="+str('0')
    header = {"Content-Type": "application/json; charset=utf-8",
              "Authorization": "Basic {enter-api-key-here}"}


    querystring = {"limit":limit, "total_count":total_count}


    # find out total number of items
    r = requests.get(url=url_address, headers=header, params=querystring).json()
    total_record = int(r['total_count'])
    print("Total record: " +str(total_record))




    # loop through all offset and return JSON object
    for offset in range(0, 100, 50):
        url = "https://onesignal.com/api/v1/notifications?app_id={enter-app-id-here}&kind=0&limit=0&offset="+str(offset)              
        response = requests.get(url=url,headers=header, params=querystring).json()        
        all_items.append(response) 
        data = json_normalize(all_items, record_path='notifications')
        print(offset)
    

    # prettify JSON
    data = json.dumps(all_items, sort_keys=True, indent=4)

    return data

print(get())

In [ ]:
##Flatten response above by using json normalize
df= pd.json_normalize(all_items)
df


In [13]:
#save response in df to a json file called notifications.json
with open("notifications.json", "w") as outfile: 
    json.dump(all_items, outfile, indent=4)


In [ ]:
#Open nested df
with open('notifications.json') as data_file:    
    data = json.load(data_file)  

df = pd.json_normalize(data, 'notifications', ['completed_at', 'converted', 'received','successful'], 
                    record_prefix='notifications_', errors='ignore')


#replace empty values with o
df=df.fillna(0)

# Convert from unix to datetime
df['notifications_completed_at'] = pd.to_datetime(df['notifications_completed_at'],unit='s')

#remove timestamp from date
df['notifications_completed_at'] = pd.to_datetime(df['notifications_completed_at']).dt.date

#Pull only specific metrics
df=df.loc[:,['notifications_completed_at','notifications_received','notifications_converted','notifications_successful']]

# save dataframe to CSV again
dfcsv=df.to_csv('notifications.csv')

#Groupby date
df=df.groupby(['notifications_completed_at']).sum()

#Create dataframe
df=pd.DataFrame(df)

#Create brandname to differentiate data from one appid to another. Here, I am creating a column called "name" and assigning it whatever name I'll decide on
df['name'] = '{name-of-column}'

#Store in csv

df=df.to_csv('notifications.csv')

#Read CSV
df=pd.read_csv('notifications.csv')

# df1=df['notifications_completed_at'].iloc[-1]
df